In [18]:
from codebase.classes_data import Data
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.ibis import essl, exp_and_normalise, run_mcmc
from codebase.plot import plot_density, plot_line, get_post_df
from tqdm.notebook import tqdm
import altair as alt

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

##

In [11]:
log_dir = './log/20201215_095024_mcmcpseudo/'
ps = load_obj('mcmc_post_samples', log_dir)

In [12]:
ps['beta'] = ps['beta'].reshape((ps['beta'].shape[0],1,ps['beta'].shape[1],ps['beta'].shape[2]))

In [14]:
ps['alpha'] = ps['alpha'].reshape((ps['alpha'].shape[0],1,ps['alpha'].shape[1]))

In [16]:
param = 'alpha'
df = get_post_df(ps[param]) 
df['source'] = 'mcmc'
plot_density(df, height=100)

alt.Chart(...)